In [1]:
import numpy as np # numpy arrays
import pandas as pd # dataframe
import seaborn as sns # plotting
import matplotlib.pyplot as plt # plotting

## pixel data collection for training finished on zooniverse

![title](notebook_imgs/1.zooniverse.png)

In [ ]:
# read training data
features  = np.load('Training_data/features.npy')/255 # normalize pixel value from 0-255 to 0-1
print(labels.shape)

labels = np.load('Training_data/labels.npy') # background, leaf, stalk , and panicle were marked by 0, 1, 2, and 3 respectively.
print(features.shape)

## visualize hyperspectral signatures of different plant parts

- preprocess data for plotting
- use matplotlib and seaborn - popular Data Visualization libraries in python

In [ ]:
# convert numpy array to pandas dataframe and normalize reflectance from 0-255 to 0-1
df = pd.DataFrame(features)
df.head()

In [ ]:
# load wavelength info. each wavelength respond to 1 band
df_wave = pd.read_csv('wavelength_band_info.txt', delim_whitespace=True)
df_wave.head()

In [ ]:
## add label data to the dataframe and rename each column in the dataframe
df.columns = df_wave['Wavelength(nm)']
df['Label'] = labels
df.head()

In [ ]:
# dataframe transformation to match the input data format of lineplot function in seaborn 
# details about lineplot https://seaborn.pydata.org/generated/seaborn.lineplot.html
df_melt = df.melt(id_vars='Label', value_name='Reflectance')
df_melt.head()

- this is what the [melt operation](https://pandas.pydata.org/docs/reference/api/pandas.melt.html) looks like:
![melt operation](notebook_imgs/melt.png)

- Now the data is ready, Let's start plotting

In [8]:
# define the colormap mapping 0,1,2,3 to different colors
from matplotlib.colors import ListedColormap
cmap = ListedColormap(['#BBBBBB', '#32CD32', '#FF8C00', '#9400D3'], name='organs') # hex code

- you can choose your color scheme in [colorbrew](https://colorbrewer2.org/#type=qualitative&scheme=Set1&n=4)
![color scheme](notebook_imgs/color_scheme.png)

In [ ]:
fig, ax = plt.subplots(figsize=(4, 2.5)) # define a figure and a single axes object
ax = sns.lineplot(ax=ax, data=df_melt, x='Wavelength(nm)', y='Reflectance',  
                  hue='Label', ci="sd", palette=cmap, linewidth=1)
ax.set_xlim(530, 1711)
ax.set_yticks([0.0, 0.2, 0.4, 0.6, 0.8])
ax.set_ylabel('Normalized Intensity', fontsize=8)
ax.set_xlabel('Wavelength(nm)', fontsize=8)
ax.legend(frameon=False, fontsize=7, labels=['background','leaf','stalk','panicle'])

ax.spines['right'].set_visible(False) # remove right axis
ax.spines['top'].set_visible(False) # remove top axis
plt.tight_layout()
plt.savefig('hyp_signature.png', dpi=300) # save figuer with resolution of 300 dpi

![images on cyverse](notebook_imgs/hyp_signature.png)

## training machine learning models

### PCA (Principal component analysis)

- unsupervised
- exploratory data analysis
- dimensionality reduction
- more details about [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html#sklearn.decomposition.PCA) on scikit-learn website

In [30]:
from sklearn.decomposition import PCA # import PCA class from scikit-learn

In [ ]:
pca = PCA(n_components=2) # let's only consider the first two PCs
pca_results = pca.fit_transform(features) # Fit the model with input and apply the dimensionality reduction on it.
print(pca_results.shape)
pca_scores = pca.explained_variance_ratio_ # variance explained by each component
print('variance explained by the first two PCs: ', pca_scores)

In [ ]:
# make scatter plot
fig, ax = plt.subplots()
s1, s2 = pca_results[:, 0], pca_results[:, 1]
scatter = ax.scatter(s1, s2, c=labels, cmap=cmap, s=20, alpha=0.7)
hs, _ = scatter.legend_elements() # handlers of the legend
ls = ['background','leaf','stalk','panicle'] # labels of the legend
ax.legend(hs, ls)
ax.set_xlabel('PC1 (%.2f)'%pca_scores[0])
ax.set_ylabel('PC2 (%.2f)'%pca_scores[1])

plt.tight_layout()
plt.savefig('pca.png', dpi=150)

![images on cyverse](notebook_imgs/pca.png)

In [34]:
from sklearn.model_selection import KFold # for five fold cross validation
from sklearn.metrics import confusion_matrix # calcuate confusion matrix

### (linear discriminant analysis)

- supervised classifier
- learn more details about [LDA](https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html) on scikit-learn website

In [35]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=123)
for train_index, test_index in kf.split(features):
    X_train, X_test = features[train_index], features[test_index] # data for training
    y_train, y_test = labels[train_index], labels[test_index] # data for validation
    
    clf = LDA() # you can specify 'n_components' option for dimensionality reduction like PCA
    clf.fit(X_train, y_train) # fit the model with both features and labels because LDA is supervised approach
    y_predict = clf.predict(X_test) # make predictions using fitted model
    acc = (y_predict==y_test).sum()/len(y_test) # calcuate the overall accuracy
    print('overal accuracy: %.3f'%acc)
    cfx = confusion_matrix(y_test, y_predict, labels=[0,1,2,3], normalize='true') # check the confusion matrix. x-axis is ground truth and y-axis is predicted
    print('accuracy in each class: ',np.diagonal(cfx))

### SVM (support vector machine)

- learn more details about [SVM](https://scikit-learn.org/stable/modules/svm.html) on scikit-learn website

In [37]:
from sklearn import svm

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=123)
for train_index, test_index in kf.split(features):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    
    clf = svm.SVC(kernel='linear') # can choose different kernels
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    acc = (y_predict==y_test).sum()/len(y_test)
    print('overal accuracy: %.3f'%acc)
    cfx = confusion_matrix(y_test, y_predict, labels=[0,1,2,3], normalize='true')
    print('accuracy in each class: ',np.diagonal(cfx))

### KNN (k-nearest neighbors)

In [39]:
from sklearn.neighbors import KNeighborsClassifier as KNN

- learn more details about [KNN](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) on scikit-learn website

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=123)
for train_index, test_index in kf.split(features):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    
    clf = KNN(n_neighbors=3) # default is 5
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    acc = (y_predict==y_test).sum()/len(y_test)
    print('overal accuracy: %.3f'%acc)
    cfx = confusion_matrix(y_test, y_predict, labels=[0,1,2,3], normalize='true')
    print('accuracy in each class: ',np.diagonal(cfx))

### random forest

- learn more details about [Random Forest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) on scikit-learn website

In [41]:
from sklearn.ensemble import RandomForestClassifier as RF

In [ ]:
ginis = []
kf = KFold(n_splits=5, shuffle=True, random_state=123)
for train_index, test_index in kf.split(features):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    
    clf = RF() # try parameters: n_estimators=200, max_depth=5
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    acc = (y_predict==y_test).sum()/len(y_test)
    print('overal accuracy: %.3f'%acc)
    cfx = confusion_matrix(y_test, y_predict, labels=[0,1,2,3], normalize='true')
    print('accuracy in each class: ',np.diagonal(cfx))
    
    gini = clf.feature_importances_ # Gini importance. The higher, the more important the feature.
    ginis.append(gini)

- check the feature importance

In [ ]:
df_gini = pd.DataFrame(ginis).transpose()
df_gini.columns = ['fold %s'%i for i in range(1,6)]
df_gini.index=df_wave['Wavelength(nm)']
df_gini.head()

In [ ]:
# plot gini importance across wavelengths
ax = df_gini.plot()
ax.set_ylabel('Gini importance')

plt.tight_layout()
plt.savefig('RF_Gini.png', dpi=150)

![images on cyverse](notebook_imgs/RF_Gini.png)

### Try Native Bayes and AdaBoost classifiers yourself

In [88]:
### Native Bayes
from sklearn.naive_bayes import GaussianNB

# put your code here

In [ ]:
## AdaBoostClassifier
from sklearn.ensemble import AdaBoostClassifier

# put your code here

## Make predictions on a hyperspectral image cube

- seems LAD performs best among all tested classifiers

In [ ]:
# train LDA model using all the training data
clf = LDA()
clf.fit(features, labels)

In [48]:
# load the hyperspectral image cube in numpy array
img_npy = np.load('NPYs/CM024_2017-08-30.npy')[35:478, :, :]/255 # remove the pot and top frame parts
img_npy.shape

(443, 320, 243)

- remember that the input dimension for our model is (N, 243) where N is the number of pixels

In [49]:
# make predictions
x, y, z = img_npy.shape
x_test = img_npy.reshape(x*y, z)
y_test = clf.predict(x_test).reshape(x, y)

In [ ]:
# show our prediction results
cmap = ListedColormap(['#FFFFFF', '#32CD32', '#FF8C00', '#9400D3'], name='test')
plt.imshow(y_test, cmap=cmap)
# plt.imsave('test.png', y_test, cmap=cmap) # run if you want to save the prediction as a png file

original vs predicted:
![images on cyverse](notebook_imgs/test_prediction1.png)

### try to make predictions on other two hyperspectral image cubes under the 'NPYs' directory

![images on cyverse](notebook_imgs/test_prediction2.png)

## Estiamte organ sizes from the predictions

- other traits requiring image processing skills will not be covered here

In [ ]:
size_leaf = (y_test==1).sum() 
size_stalk = (y_test==2).sum()
size_panicle = (y_test==3).sum()
size_leaf, size_stalk, size_panicle

## Hyperspectral data in numpy format and Genotype data in hmp format for a sorghum diverse population

- Download all image data from [Cyverse](https://datacommons.cyverse.org/browse/iplant/home/shared/commons_repo/curated/Miao_Schnable_sorghumHighThroughputPhenotyping_2017)

![images on cyverse](notebook_imgs/cyverse.png)

- Download genotype data from [figshare](https://doi.org/10.6084/m9.figshare.11462469.v5)
![figshare genotype](notebook_imgs/figshare.png)

## To cite

Cite this paper if you use the hyperspectral image data for the sorghum association panel:
- Chenyong Miao, Alejandro Pages, Zheng Xu, Eric Rodene, Jinliang Yang, and James C. Schnable (2020) Semantic segmentation of sorghum using hyperspectral data identifies genetic associations. **Plant Phenomics** doi: 10.34133/2020/4216373

Please cite this paper if you use the genotype data for the same population
- Chenyong Miao, Yuhang Xu, Sanzhen Liu, Patrick S. Schnable, and James C. Schnable (2020) Increased power and accuracy of locus identification in time-series genome-wide association in sorghum. **Plant Physiology** doi: 10.1101/2020.02.16.951467